In [ ]:
import re
import string
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import pytorch_lightning as pl
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger
from torchmetrics.classification import (MultilabelF1Score, MultilabelPrecision, 
                                         MultilabelRecall, MultilabelHammingDistance, 
                                         MultilabelJaccardIndex)
from sklearn.metrics import classification_report
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/daniellai/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [41]:
# 1. Data Loading and Preprocessing
# Load your data – adjust the path and method (read_csv or read_parquet) as needed.
data_full = pd.read_parquet("/Users/daniellai/MSDS_2026/MSDS_2024_2026/Winter_2025/DSCI521/Project/data/data_final.parquet")
data_full.head()

,title,authors,summary,published,updated,link,pdf_url,categories,target
0,Is Physics Sick? [In Praise of Classical Physics],Hisham Ghassib,"In this paper, it is argued that theoretical p...",2012-09-04T10:32:56Z,2012-09-04T10:32:56Z,http://arxiv.org/abs/1209.0592v1,http://arxiv.org/pdf/1209.0592v1,"physics.gen-ph, physics.hist-ph",physic
1,Modern Mathematical Physics: what it should be?,Ludwig Faddeev,Personal view of author on goals and content o...,2000-02-08T13:13:00Z,2000-02-10T10:14:56Z,http://arxiv.org/abs/math-ph/0002018v2,http://arxiv.org/pdf/math-ph/0002018v2,"math-ph, hep-th, math.MP","math-stats,physic"
2,Topology in Physics,R. Jackiw,The phenomenon of quantum number fractionaliza...,2005-03-15T16:00:59Z,2005-03-15T16:00:59Z,http://arxiv.org/abs/math-ph/0503039v1,http://arxiv.org/pdf/math-ph/0503039v1,"math-ph, cond-mat.mes-hall, math.MP, physics.c...","math-stats,physic"
3,Contents of Physics Related E-Print Archives,"E. R. Prakasan, Anil Kumar, Anil Sagar, Lalit ...",The frontiers of physics related e-print archi...,2003-08-28T13:12:57Z,2003-08-28T13:12:57Z,http://arxiv.org/abs/physics/0308107v1,http://arxiv.org/pdf/physics/0308107v1,physics.data-an,physic
4,Fundamental Dilemmas in Theoretical Physics,Hisham Ghassib,"In this paper, we argue that there are foundat...",2014-05-22T07:49:09Z,2014-05-22T07:49:09Z,http://arxiv.org/abs/1405.5530v1,http://arxiv.org/pdf/1405.5530v1,physics.hist-ph,physic


In [42]:
data_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150171 entries, 0 to 150170
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   title       150171 non-null  object
 1   authors     150171 non-null  object
 2   summary     150171 non-null  object
 3   published   150171 non-null  object
 4   updated     150171 non-null  object
 5   link        150171 non-null  object
 6   pdf_url     150171 non-null  object
 7   categories  150171 non-null  object
 8   target      150171 non-null  object
dtypes: object(9)
memory usage: 10.3+ MB


In [ ]:
# normalizing the text 
wpt = nltk.WordPunctTokenizer()
def normalized_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'<.*?>+', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', ' ', text)
    tokens = wpt.tokenize(text)
    cleaned_text = ' '.join(tokens)

    return cleaned_text

In [44]:
data_full['full_title'] = data_full['summary'].apply(normalized_text)
data_full.head()

,title,authors,summary,published,updated,link,pdf_url,categories,target,full_title
0,Is Physics Sick? [In Praise of Classical Physics],Hisham Ghassib,"In this paper, it is argued that theoretical p...",2012-09-04T10:32:56Z,2012-09-04T10:32:56Z,http://arxiv.org/abs/1209.0592v1,http://arxiv.org/pdf/1209.0592v1,"physics.gen-ph, physics.hist-ph",physic,in this paper it is argued that theoretical ph...
1,Modern Mathematical Physics: what it should be?,Ludwig Faddeev,Personal view of author on goals and content o...,2000-02-08T13:13:00Z,2000-02-10T10:14:56Z,http://arxiv.org/abs/math-ph/0002018v2,http://arxiv.org/pdf/math-ph/0002018v2,"math-ph, hep-th, math.MP","math-stats,physic",personal view of author on goals and content o...
2,Topology in Physics,R. Jackiw,The phenomenon of quantum number fractionaliza...,2005-03-15T16:00:59Z,2005-03-15T16:00:59Z,http://arxiv.org/abs/math-ph/0503039v1,http://arxiv.org/pdf/math-ph/0503039v1,"math-ph, cond-mat.mes-hall, math.MP, physics.c...","math-stats,physic",the phenomenon of quantum number fractionaliza...
3,Contents of Physics Related E-Print Archives,"E. R. Prakasan, Anil Kumar, Anil Sagar, Lalit ...",The frontiers of physics related e-print archi...,2003-08-28T13:12:57Z,2003-08-28T13:12:57Z,http://arxiv.org/abs/physics/0308107v1,http://arxiv.org/pdf/physics/0308107v1,physics.data-an,physic,the frontiers of physics related e print archi...
4,Fundamental Dilemmas in Theoretical Physics,Hisham Ghassib,"In this paper, we argue that there are foundat...",2014-05-22T07:49:09Z,2014-05-22T07:49:09Z,http://arxiv.org/abs/1405.5530v1,http://arxiv.org/pdf/1405.5530v1,physics.hist-ph,physic,in this paper we argue that there are foundati...


In [45]:
# Split data: 80% train, 10% validation, 10% test
train_val_df, test_df = train_test_split(data_full, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.1, random_state=42)


In [46]:

train_texts = train_df['full_title'].tolist()
train_targets = train_df['target'].tolist()
val_texts = val_df['full_title'].tolist()
val_targets = val_df['target'].tolist()
test_texts = test_df['full_title'].tolist()
test_targets = test_df['target'].tolist()

In [47]:
# Build label mapping from the entire dataset
all_targets = data_full['target'].tolist()
all_labels = set()
for target in all_targets:
    all_labels.update(target.split(','))
label_to_idx = {label.strip(): idx for idx, label in enumerate(sorted(all_labels))}
print("Label Mapping:", label_to_idx)

Label Mapping: {'bio': 0, 'cs': 1, 'econ-qfin': 2, 'eess': 3, 'math-stats': 4, 'physic': 5}


In [48]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
max_length = 256

In [49]:
class SciBERTMultiLabelDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, label_to_idx, max_length=256):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.label_to_idx = label_to_idx
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        # Remove the batch dimension.
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        
        # Convert target string (e.g., "cs,bio") to a multi-hot vector.
        target_str = self.targets[idx]
        label_vec = torch.zeros(len(self.label_to_idx), dtype=torch.float)
        for label in target_str.split(','):
            label = label.strip()
            if label in self.label_to_idx:
                label_vec[self.label_to_idx[label]] = 1
        item['labels'] = label_vec
        return item


In [50]:
# Instantiate datasets.
train_dataset = SciBERTMultiLabelDataset(train_texts, train_targets, tokenizer, label_to_idx, max_length)
val_dataset = SciBERTMultiLabelDataset(val_texts, val_targets, tokenizer, label_to_idx, max_length)
test_dataset = SciBERTMultiLabelDataset(test_texts, test_targets, tokenizer, label_to_idx, max_length)

In [51]:
# 4. Handle Imbalance with WeightedRandomSampler
def compute_sample_weight(label_vec):
    return 1.0 / (label_vec.sum().item() + 1e-6)

sample_weights = [compute_sample_weight(train_dataset[i]['labels']) for i in range(len(train_dataset))]
sample_weights = np.array(sample_weights)
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

In [52]:
def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.stack([item["labels"] for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [53]:
train_loader = DataLoader(train_dataset, batch_size=16, sampler=sampler, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma= 3 , alpha=0.75, reduction='mean'):
        """
        gamma: Focusing parameter.
        alpha: Balancing parameter.
        reduction: 'mean', 'sum', or 'none'
        """
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, logits, targets):
        # Compute binary cross-entropy loss without reduction
        bce_loss = nn.functional.binary_cross_entropy_with_logits(logits, targets, reduction='none')
        probs = torch.sigmoid(logits)
        p_t = targets * probs + (1 - targets) * (1 - probs)
        # Compute modulating factor
        focal_factor = (1 - p_t) ** self.gamma
        loss = focal_factor * bce_loss
        # Apply the alpha balancing factor
        if self.alpha is not None:
            if isinstance(self.alpha, (float, int)):
                alpha_factor = targets * self.alpha + (1 - targets) * (1 - self.alpha)
            else:
                alpha_factor = targets * self.alpha + (1 - targets) * (1 - self.alpha)
            loss = alpha_factor * loss
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

In [ ]:
class SciBERTMultiLabel(pl.LightningModule):
    def __init__(self, num_labels, learning_rate=0.001, test_threshold=0.45):
        super(SciBERTMultiLabel, self).__init__()
        
        # Update the configuration to use dropout=0.6.
        config = AutoConfig.from_pretrained(
            "allenai/scibert_scivocab_uncased", 
            num_labels=num_labels, 
            problem_type="multi_label_classification",
            hidden_dropout_prob=0.4,
            attention_probs_dropout_prob=0.4
        )
        self.model = AutoModelForSequenceClassification.from_pretrained(
            "allenai/scibert_scivocab_uncased", 
            config=config
        )
        self.learning_rate = learning_rate
        self.test_threshold = test_threshold
        
        # Use Focal Loss.
        self.criterion = FocalLoss(gamma=3, alpha=0.75, reduction='mean')
        self.f1 = MultilabelF1Score(num_labels=num_labels, average='macro')
        self.precision = MultilabelPrecision(num_labels=num_labels, average='macro')
        self.recall = MultilabelRecall(num_labels=num_labels, average='macro')
        self.hamming = MultilabelHammingDistance(num_labels=num_labels)
        self.jaccard = MultilabelJaccardIndex(num_labels=num_labels)
        
        # Initialize lists to store predictions and labels for classification report.
        self.test_preds = []
        self.test_labels = []
        
    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits
    
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        logits = self(input_ids, attention_mask)
        loss = self.criterion(logits, labels)
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        logits = self(input_ids, attention_mask)
        loss = self.criterion(logits, labels)
        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        logits = self(input_ids, attention_mask)
        loss = self.criterion(logits, labels)
        self.log("test_loss", loss, on_epoch=True)
        
        probs = torch.sigmoid(logits)
        preds = (probs > self.test_threshold).float()
        
        # Update torchmetrics.
        self.f1.update(preds, labels)
        self.precision.update(preds, labels)
        self.recall.update(preds, labels)
        self.hamming.update(preds, labels)
        self.jaccard.update(preds, labels)
        
        # Save predictions and labels for the classification report.
        self.test_preds.append(preds.detach().cpu())
        self.test_labels.append(labels.detach().cpu())
        
        return loss
    
    def on_test_epoch_end(self):
        f1 = self.f1.compute()
        precision = self.precision.compute()
        recall = self.recall.compute()
        hamming = self.hamming.compute()
        jaccard = self.jaccard.compute()
        
        print("\n=== Test Metrics ===")
        print(f"Test F1 Score: {f1:.4f}")
        print(f"Test Precision: {precision:.4f}")
        print(f"Test Recall: {recall:.4f}")
        print(f"Test Hamming Loss: {hamming:.4f}")
        print(f"Test Jaccard Score: {jaccard:.4f}")
        
        # Reset metrics.
        self.f1.reset()
        self.precision.reset()
        self.recall.reset()
        self.hamming.reset()
        self.jaccard.reset()
        
        # Concatenate all predictions and labels.
        all_preds = torch.cat(self.test_preds, dim=0).numpy()
        all_labels = torch.cat(self.test_labels, dim=0).numpy()
        
        # Generate and print a detailed classification report.
        report = classification_report(all_labels, all_preds, output_dict=True)
        print("\n=== Classification Report ===")
        for label, metrics in report.items():
            print(f"{label}: {metrics}")
        
        # Clear stored predictions and labels.
        self.test_preds = []
        self.test_labels = []
    
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

In [56]:
csv_logger = CSVLogger("logs", name="scibert_multilabel")
tb_logger = TensorBoardLogger("logs", name="scibert_multilabel_tensorboard")

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=1,
    verbose=True,
    mode="min"
)
lr_monitor = LearningRateMonitor(logging_interval="epoch")

trainer = pl.Trainer(
    max_epochs=4,
    accelerator="mps" if torch.backends.mps.is_available() else "cpu",
    devices=1,
    logger=[csv_logger, tb_logger],
    log_every_n_steps=10,
    callbacks=[early_stop_callback, lr_monitor]
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
model = SciBERTMultiLabel(num_labels=len(label_to_idx), learning_rate=0.001, test_threshold=0.45)
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

  | Name      | Type                          | Params | Mode 
--------------------------------------------------------------------
0 | model     | BertForSequenceClassification | 109 M  | eval 
1 | criterion | FocalLoss                     | 0      | train
2 | f1        | MultilabelF1Score             | 0      | train
3 | precision | MultilabelPrecision           | 0      | train
4 | recall    | MultilabelRecall              | 0      | train
5 | hamming   | MultilabelHammingDistance     | 0      | train
6 | jaccard   | MultilabelJaccardIndex        | 0      | train
--------------------------------------------------------------------
109 M     Trainable params
0         Non-trainabl

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/opt/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Testing: |          | 0/? [00:00<?, ?it/s]


=== Test Metrics ===
Test F1 Score: 0.7891
Test Precision: 0.8165
Test Recall: 0.7660
Test Hamming Loss: 0.0709
Test Jaccard Score: 0.6603

=== Classification Report ===
0: {'precision': 0.7837552742616034, 'recall': 0.7234664070107109, 'f1-score': 0.7524050632911392, 'support': 1027.0}
1: {'precision': 0.9090399701158013, 'recall': 0.8363980065303317, 'f1-score': 0.8712073749216862, 'support': 5819.0}
2: {'precision': 0.8871745419479267, 'recall': 0.7165109034267912, 'f1-score': 0.7927617406290393, 'support': 1284.0}
3: {'precision': 0.6366704161979753, 'recall': 0.5895833333333333, 'f1-score': 0.6122228231476474, 'support': 960.0}
4: {'precision': 0.8048951048951049, 'recall': 0.8435324294613411, 'f1-score': 0.8237609590266595, 'support': 5458.0}
5: {'precision': 0.877602752127467, 'recall': 0.8864301389904902, 'f1-score': 0.8819943590210172, 'support': 5468.0}
micro avg: {'precision': 0.8498279492578706, 'recall': 0.8266886490807354, 'f1-score': 0.8380986147339631, 'support': 20016

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.013523343950510025    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.013523343950510025}]

In [ ]:
def predict_sci_bert(sentence, tokenizer, label_to_idx, model, threshold=0.45, max_length=256):
    model.eval()
    with torch.no_grad():
        encoding = tokenizer(
            sentence,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        input_ids = encoding["input_ids"]
        attention_mask = encoding["attention_mask"]
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.sigmoid(logits)
        preds = (probs > threshold).float().squeeze(0)
        predicted_labels = [label for label, idx in label_to_idx.items() if preds[idx] == 1]
        return predicted_labels

In [72]:
example_sentences = [
    "Attention is all you need The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.",
    "Vector autogressions (VARs) are widely applied when it comes to modeling and forecasting macroeconomic variables. In high dimensions, however, they are prone to overfitting. Bayesian methods, more concretely shrinkage priors, have shown to be successful in improving prediction performance. In the present paper, we introduce the semi-global framework, in which we replace the traditional global shrinkage parameter with group-specific shrinkage parameters. We show how this framework can be applied to various shrinkage priors, such as global-local priors and stochastic search variable selection priors. We demonstrate the virtues of the proposed framework in an extensive simulation study and in an empirical application forecasting data of the US economy. Further, we shed more light on the ongoing ``Illusion of Sparsity'' debate, finding that forecasting performances under sparse/dense priors vary across evaluated economic variables and across time frames. Dynamic model averaging, however, can combine the merits of both worlds.",
    "Classification can be performed using either a discriminative or a generative learning approach. Discriminative learning consists of constructing the conditional probability of the outputs given the inputs, while generative learning consists of constructing the joint probability density of the inputs and outputs. Although most classical and quantum methods are discriminative, there are some advantages of the generative learning approach. For instance, it can be applied to unsupervised learning, statistical inference, uncertainty estimation, and synthetic data generation. In this article, we present a quantum generative multiclass classification strategy, called quantum generative classification (QGC). This model uses a variational quantum algorithm to estimate the joint probability density function of features and labels of a data set by means of a mixed quantum state. We also introduce a quantum map called quantum-enhanced Fourier features (QEFF), which leverages quantum superposition to prepare high-dimensional data samples in quantum hardware using a small number of qubits. We show that the quantum generative classification algorithm can be viewed as a Gaussian mixture that reproduces a kernel Hilbert space of the training data. In addition, we developed a hybrid quantum-classical neural network that shows that it is possible to perform generative classification on high-dimensional data sets. The method was tested on various low- and high-dimensional data sets including the 10-class MNIST and Fashion-MNIST data sets, illustrating that the generative classification strategy is competitive against other previous quantum models.",
    "Research on human skin anatomy reveals its complex multi-scale, multi-phase nature, with up to 70% of its composition being bounded and free water. Fluid movement plays a key role in the skin's mechanical and biological responses, influencing its time-dependent behavior and nutrient transport.Poroelastic modeling is a promising approach for studying skin dynamics across scales by integrating multi-physics processes. This paper introduces a hierarchical two-compartment model capturing fluid distribution in the interstitium and micro-circulation. A theoretical framework is developed with a biphasic interstitium -- distinguishing interstitial fluid and non-structural cells -- and analyzed through a one-dimensional consolidation test of a column. This biphasic approach allows separate modeling of cell and fluid motion, considering their differing characteristic times. An appendix discusses extending the model to include biological exchanges like oxygen transport. Preliminary results indicate that cell viscosity introduces a second characteristic time, and at high viscosity and short time scales, cells behave similarly to solids.A simplified model was used to replicate an experimental campaign on short time scales. Local pressure (up to 31 kPa) was applied to dorsal finger skin using a laser Doppler probe PF801 (Perimed Sweden), following a setup described in Fromy Brain Res (1998). The model qualitatively captured ischemia and post-occlusive reactive hyperemia, aligning with experimental data.All numerical simulations used the open-source software FEniCSx v0.9.0. To ensure transparency and reproducibility, anonymized experimental data and finite element codes are publicly available on GitHub.",
    "Currency arbitrage capitalizes on price discrepancies in currency exchange rates between markets to produce profits with minimal risk. By employing a combinatorial optimization problem, one can ascertain optimal paths within directed graphs, thereby facilitating the efficient identification of profitable trading routes. This research investigates the methodologies of quantum annealing and gate-based quantum computing in relation to the currency arbitrage problem. In this study, we implement the Quantum Approximate Optimization Algorithm (QAOA) utilizing Qiskit version 1.2. In order to optimize the parameters of QAOA, we perform simulations utilizing the AerSimulator and carry out experiments in simulation. Furthermore, we present an NchooseK-based methodology utilizing D-Wave's Ocean suite. This methodology enables a comparison of the effectiveness of quantum techniques in identifying optimal arbitrage paths. The results of our study enhance the existing literature on the application of quantum computing in financial optimization challenges, emphasizing both the prospective benefits and the present limitations of these developing technologies in real-world scenarios.",
    "Despite advances in methods to interrogate tumor biology, the observational and population-based approach of classical cancer research and clinical oncology does not enable anticipation of tumor outcomes to hasten the discovery of cancer mechanisms and personalize disease management. To address these limitations, individualized cancer forecasts have been shown to predict tumor growth and therapeutic response, inform treatment optimization, and guide experimental efforts. These predictions are obtained via computer simulations of mathematical models that are constrained with data from a patient's cancer and experiments. This book chapter addresses the validation of these mathematical models to forecast tumor growth and treatment response. We start with an overview of mathematical modeling frameworks, model selection techniques, and fundamental metrics. We then describe the usual strategies employed to validate cancer forecasts in preclinical and clinical scenarios. Finally, we discuss existing barriers in validating these predictions along with potential strategies to address them."]

In [ ]:
print("\n=== SciBERT Back-Testing Results ===")
for sentence in example_sentences:
    predicted_labels = predict_sci_bert(sentence, tokenizer, label_to_idx, model, threshold=0.45)
    print(f"Sentence: {sentence[:80]}...")
    print(f"Predicted Labels: {predicted_labels}\n")


=== SciBERT Back-Testing Results ===
Sentence: Attention is all you need The dominant sequence transduction models are based on...
Predicted Labels: ['cs']

Sentence: Vector autogressions (VARs) are widely applied when it comes to modeling and for...
Predicted Labels: ['math-stats']

Sentence: Classification can be performed using either a discriminative or a generative le...
Predicted Labels: ['cs', 'math-stats', 'physic']

Sentence: Research on human skin anatomy reveals its complex multi-scale, multi-phase natu...
Predicted Labels: ['bio', 'physic']

Sentence: Currency arbitrage capitalizes on price discrepancies in currency exchange rates...
Predicted Labels: ['econ-qfin', 'physic']

Sentence: Despite advances in methods to interrogate tumor biology, the observational and ...
Predicted Labels: ['bio', 'math-stats']

